# About
### Data
Data was sourced from https://climate.colostate.edu/data_access_new.html. The stations looked at in the current dataset is monthly data for the ASPEN 1SW, STEAMBOAT SPRINGS, BRECKENRIDGE, and TELLURIDE 4WNW. 

### Library Dependencies
- pandas
- altair
- pathlib

In [1]:
# Import libraries
import pandas as pd
import altair as alt
from pathlib import Path

# Data
### Importing
All data is stored in a local directory titled "data". The "import_CSU_data" function takes in this directory and imports all CSV files into pandas dataframes. These dataframes are merged into a single dataframe with an additional field created to denote the weather station. 

#### Cleaning
Upon examining the data, there are months for which no snowfall data was collected. In order to not count this data when aggregating, a field called "complete" was created to denote whether that year for that station has snowfall data for each month. 


Additional changes:
- All fields originial to the CSV data (tmax, tmin, precip, and snowfall) we're converted to numeric data types.
- Created "season" field for additional grouping options
- Created "year" field for ease of plotting



In [2]:
# Load the data
def import_CSU_data(data_dir):

    dfs = []

    for filename in data_dir.iterdir():
        with open(filename, "r") as f:
            if filename.name.startswith("."):
                continue
            station = f.readline().strip()
            df = pd.read_csv(
                f, header=None, names=["date", "tmax", "tmin", "precip", "snowfall"]
            )

            df["station"] = station
            df["date"] = pd.to_datetime(df["date"])
            df = df[df["date"].dt.year < 2025]

        dfs.append(df)

    df = pd.concat(dfs)

    numeric_cols = ["tmax", "tmin", "precip", "snowfall"]

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    return df

base_dir = Path().resolve()
data_dir = base_dir / "data"
data = import_CSU_data(data_dir)

In [3]:
# Convert Data Types
numeric_cols = ["tmax", "tmin", "precip", "snowfall"]

for col in numeric_cols:
    data[col] = pd.to_numeric(data[col], errors="coerce")

# Add Year Column
data["year"] = data["date"].dt.year

# Add Season Column
def month_to_season(month):
    if month in [12, 1, 2]: return "winter"
    if month in [3, 4, 5]: return "spring"
    if month in [6, 7, 8]: return "summer"
    else: return "fall"

data["season"] = data["date"].dt.month.apply(month_to_season)


# Count missing months per year per station
complete_years = (
    data.groupby(["station", "year"])
    .snowfall
    .apply(lambda x: x.notna().all())  # True if all months have data
    .reset_index(name="complete")
)
data_clean = data.merge(complete_years, on=["station", "year"])

# Visualization
### Common Plotting Options
Created variables to store commonly used plotting options including...
- color schemes
- line chart options
- scatter point options
- chart sizing options
- tick mark selection

### Visualization 1
The first visualization looks at yearly snowfall trends for each station. Users can hover over a year to show the monthly snowfall trends for that year for each station.

### Visualization 2
The second visualization looks at variability in snowfall. The violin plot shows the distribution of yearly snowfall by station. Users can select a specific violin to get a more detailed view at the variation in snowfall for that station. The bottom-left visual shows the percentage difference of that years snowfall vs the average snowfall for that station. The bottom-right visual shows the total average as well as a five year rolling average of yearly snowfall for the selected station. 

In [18]:
pastel2_plus = [
    "#66C2A5",
    "#FC8D62",
    "#8DA0CB",
    "#E78AC3",
    "#A6D854",
    "#FFD92F",
    "#E5C494",
    "#B3B3B3",
]

tick_values = [int(y) for y in sorted(data_clean["year"].unique()) if y % 5 == 0]

chart_opts = {"width":600, "height":200}
point_opts = {"size":50}
line_opts = {"strokeWidth":5, "interpolate":"monotone", "opacity":0.5}

In [19]:

selection_year = alt.selection_point(
    fields=["year"],
    nearest=True,
    on = "mouseover",
    clear = "mouseout",
    empty=False)

color_year = alt.Color(
    "station:N",
    title="Weather Station",
    scale=alt.Scale(range=pastel2_plus)
)

# --- Year chart ---
year_base = (
    alt.Chart(data_clean)
    .encode(
        x=alt.X(
            "year:O",
            title="Year",
            axis=alt.Axis(
                labelAngle=0,
                values= tick_values
            )
        ),
        y=alt.Y(
            "sum(snowfall):Q",
            title="Snowfall per Year (in)",
            scale=alt.Scale(domain=[0, 350])
            ),
        color=color_year
    )
    .transform_filter("datum.complete == true")
    .properties(**chart_opts)
)

year_point = year_base.mark_circle(**point_opts).add_params(selection_year).encode(tooltip=["year:O"])
year_line  = year_base.mark_line(**line_opts)
year_rule = (
    year_base
    .mark_rule(color="gray", strokeWidth=1)
    .encode(
        opacity=alt.condition(
            selection_year,
            alt.value(0.5),   # visible on hover
            alt.value(0)      # invisible otherwise
        ),
    )
)

year = year_point + year_line + year_rule

# --- Month chart ---
month_base = (
    alt.Chart(data_clean)
    .encode(
        x=alt.X(
            "date:T",
            timeUnit = "month",
            title = "Month"
        ),
        y=alt.Y(
            "snowfall:Q",
            title="Snowfall per Month (in)"
        ),
        color=color_year
    )
    .transform_filter(selection_year)
    .transform_filter("datum.complete == true")
    .properties(**chart_opts)
)

month_point = month_base.mark_circle(**point_opts)
month_line  = month_base.mark_line(**line_opts)
month = month_point + month_line

# --- Stack vertically with fixed size ---
chart = year & month
chart


alt.VConcatChart(...)

In [7]:
filter = data_clean["complete"] == True
avg_years = data_clean[filter].groupby(["station", "year"]).agg(
    {
        "snowfall" : "sum"
    }
)
data_year = avg_years.groupby(level="station").apply(
    lambda x: x.assign(
        five_year_avg = x["snowfall"].rolling(5).mean(),
        total_avg = x["snowfall"].mean()
    )
).droplevel(0).reset_index()

data_year["diff_from_five"] = (data_year["snowfall"] - data_year["five_year_avg"])/data_year["five_year_avg"] * 100
data_year["diff_from_total"] = (data_year["snowfall"] - data_year["total_avg"])/data_year["total_avg"] * 100

In [20]:
selection_ = alt.selection_point(
    fields=["station"],
    empty=False
)

color_ = alt.Color(
    "station:N",
    title="Weather Station",
    scale=alt.Scale(range=pastel2_plus)
)

raw_base = (
    alt.Chart(data_year)
    .encode(
        alt.X("year:O", title="Year"),
        alt.Y("snowfall:Q", title="Snowfall (in)")
    )
)

raw_point = raw_base.mark_point()
raw_line = raw_base.mark_line(interpolate="monotone")
raw = raw_point + raw_line

violin = (
    alt.Chart(data_year)
    .transform_density(
        "snowfall",
        as_=["snowfall", "density"],
        groupby=["station"]
    )
    .mark_area(orient="horizontal")
    .encode(
        x = alt.X("density:Q", stack="center", impute=None, title=None)
                .axis(labels=False, values=[0], grid=False, ticks=True),
        y = alt.Y("snowfall:Q", title="Snowfall (in)"),
        color = color_,
        column = alt.Column("station:N").spacing(0).header(labels=False, title=None)
    )
    .add_params(selection_)
    .properties(width = 160, height = 200)
)

diff_bar = (
    alt.Chart(data_year)
    .mark_bar()
    .encode(
        x = alt.X("year:O", title = "Year", axis=alt.Axis(
                labelAngle=0,
                values= tick_values
            )),
        y = alt.Y("diff_from_total", title=None),
        xOffset = "station:N",
        color = color_
    )
    .transform_filter(selection_)
    .properties(title="Percentage Difference from Average", width = 300, height = 200)
)

diff_line_base = (
    alt.Chart(data_year)
    .encode(
        x = alt.X("year:O", title="Year", axis=alt.Axis(
                labelAngle=0,
                values= tick_values
            )),
        color = color_
    )
    .transform_filter(selection_)
    .properties(width = 300, height = 200)
)
diff_line_total = diff_line_base.mark_line().encode(
    y = alt.Y("total_avg:Q", title=None, scale=alt.Scale(domain=[0, 250]))
).properties(title="Long Term vs Five Year Rolling Average")

diff_line_5_year = diff_line_base.mark_line(strokeDash=[2, 1], point=True).encode(
    y = alt.Y("five_year_avg:Q", title=None)
)

diff_line = alt.layer(diff_line_total, diff_line_5_year)
diff_line.properties(**chart_opts)

diff = alt.hconcat(diff_bar,diff_line)

alt.vconcat(violin, diff)

alt.VConcatChart(...)